In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('./data/zbiór_5.csv')

In [3]:
df.head()

,szczegolnaFormaPrawna_Symbol,formaWlasnosci_Symbol,pkdKod,wsk_liczba_dni_istnienia,Aktywa,Aktywa_trwale,Wartosci_niematerialne_prawne,Wartosc_firmy,Rzeczowe_aktywa_trwale,Srodki_trwale,...,wsk_struktura_kap_wlasnego_s_1,wsk_struktura_kap_wlasnego_s_2,wsk_zadluzenia,wsk_zob_dlugoterminowe_aktywa_rzeczowe,wsk_zob_oprocentowanych,wsk_zob_oprocentowanych_aktywa_rzeczowe,wsk_struktura_kap_obcego_s,wsk_zob_s_aktywa_rzeczowe,wsk_fin_majatku_kapitalem,default
0,117,214,4671,2647,196590.96,0.00,0.0,0.0,0.0,0.0,...,0.183009,5.291262,4.464217,0.000000,0.508670,0.508670,0.816991,-0.148422,0.183009,1
1,117,214,2849,2066,5000.00,0.00,0.0,0.0,0.0,0.0,...,1.000000,inf,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,1.000000,1
2,117,214,7022,3163,5937.59,0.00,0.0,0.0,0.0,0.0,...,1.000000,inf,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,1.000000,1
3,117,214,6312,1947,281108.61,2262.86,0.0,0.0,0.0,0.0,...,0.161707,0.161707,5.184009,0.838293,0.613571,0.613571,0.838293,0.838293,0.154905,1
4,117,214,6201,2586,1961.43,0.00,0.0,0.0,0.0,0.0,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1


In [4]:
df['default'].value_counts()

default
0    2822
1     178
Name: count, dtype: int64